In [1]:
from collections import defaultdict
import numpy as np

# noinspection SpellCheckingInspection
class NaiveBayesClassifier(object):
    def __init__(self, n_gram=1, printing=False):
        self.prior = defaultdict(int)
        self.logprior = {}
        self.bigdoc = defaultdict(list)
        self.loglikelihoods = defaultdict(defaultdict)
        self.V = []
        self.n = n_gram

    def compute_prior_and_bigdoc(self, training_set, training_labels):
        '''
        Computes the prior and the bigdoc (from the book's algorithm)
        :param training_set:
            a list of all documents of the training set
        :param training_labels:
            a list of labels corresponding to the documents in the training set
        :return:
            None
        '''
        for x, y in zip(training_set, training_labels):
            all_words = x.split(" ")
            if self.n == 1:
                grams = all_words
            else:
                grams = self.words_to_grams(all_words)

            self.prior[y] += len(grams)
            self.bigdoc[y].append(x)

    def compute_vocabulary(self, documents):
        vocabulary = set()

        for doc in documents:
            for word in doc.split(" "):
                vocabulary.add(word.lower())

        return vocabulary

    def count_word_in_classes(self):
        counts = {}
        for c in list(self.bigdoc.keys()):
            docs = self.bigdoc[c]
            counts[c] = defaultdict(int)
            for doc in docs:
                words = doc.split(" ")
                for word in words:
                    counts[c][word] += 1

        return counts

    def train(self, training_set, training_labels, alpha=1):
        # Get number of documents
        N_doc = len(training_set)

        # Get vocabulary used in training set
        self.V = self.compute_vocabulary(training_set)

        # Create bigdoc
        for x, y in zip(training_set, training_labels):
            self.bigdoc[y].append(x)

        # Get set of all classes
        all_classes = set(training_labels)

        # Compute a dictionary with all word counts for each class
        self.word_count = self.count_word_in_classes()

        # For each class
        for c in all_classes:
            # Get number of documents for that class
            N_c = float(sum(training_labels == c))

            # Compute logprior for class
            self.logprior[c] = np.log(N_c / N_doc)

            # Calculate the sum of counts of words in current class
            total_count = 0
            for word in self.V:
                total_count += self.word_count[c][word]

            # For every word, get the count and compute the log-likelihood for this class
            for word in self.V:
                count = self.word_count[c][word]
                self.loglikelihoods[c][word] = np.log((count + alpha) / (total_count + alpha * len(self.V)))

    def predict(self, test_doc):
        sums = {
            0: 0,
            1: 0,
        }
        for c in self.bigdoc.keys():
            sums[c] = self.logprior[c]
            words = test_doc.split(" ")
            for word in words:
               if word in self.V:
                   sums[c] += self.loglikelihoods[c][word]

        return sums


In [2]:
doc1 = "just plain boring"                      # -
doc2 = "entirely predictable and lacks energy"  # -
doc3 = "no surprises and very few laughs"       # -
doc4 = "very powerful"                          # +
doc5 = "the most fun film of the summer"        # +

In [3]:
training_set = [doc1, doc2, doc3, doc4, doc5]
training_labels = np.array([0, 0, 0, 1 ,1])

In [5]:
doc6 = "predictable with no fun" # ?
#
NBclassifier = NaiveBayesClassifier(n_gram=1)
NBclassifier.train(training_set,training_labels)
#
result = NBclassifier.predict(doc6)

print(result)


{0: -9.703612836494585, 1: -10.325031041273633}
